In [ ]:
import pandas as pd
from sys import argv
#import sqlalchemy
import numpy as np

#from sklearn import *
from sklearn.ensemble import RandomForestClassifier
import joblib


#import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras.layers import Flatten

#import keras

import os

import matplotlib.pyplot as plt

from datetime import datetime

In [ ]:
from sklearn.inspection import permutation_importance
import rfpimp

In [ ]:
def sql_query_to_data_frame(query):
    connection = engine.connect()
    df = pd.read_sql_query(sqlalchemy.text(query), connection)
    connection.close()
    return df

In [ ]:
#os.chdir("/home/lindali/Documents/DPhil studies/thesis_work/")

In [ ]:
def generate_features(all_df, create_param = 'created_at'):
    created_at_dt = pd.to_datetime(all_df[create_param], format = "%a %b %d %H:%M:%S +0000 %Y ", errors = 'coerce')
    created_at_dt_new = pd.to_datetime(all_df[create_param], format = "%a %b %d %H:%M:%S +0000 %Y", errors = 'coerce')
    created_at_dt = [created_at_dt_new[i] if pd.isnull(created_at_dt[i]) else created_at_dt[i] for i in range(len(created_at_dt))]
    all_df['created_at_dt'] = created_at_dt
    all_df['user_age'] = [datetime.now() - i for i in all_df['created_at_dt']]
    all_df['user_age'] = [i.total_seconds()//3600 for i in all_df['user_age']]
    main_traits = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 'listed_count']
    derived_traits = ['tweet_freq','followers_growth','friends_growth', 'favourites_growth', 'listed_growth']

    for i in range(5):
        all_df[main_traits[i]] = [int(k) for k in all_df[main_traits[i]]]
        all_df[derived_traits[i]] = all_df[main_traits[i]]/all_df['user_age']

    all_df['follower_friend_ratio'] = all_df['followers_count']/(all_df['friends_count'] + 1)
    all_df['screen_name_length'] = [len(i) for i in all_df['screen_name']]
    all_df['digits_in_screen_name'] = [sum(c.isdigit() for c in i) for i in all_df['screen_name']]
    all_df['name_length'] = [len(i) if pd.notnull(i) else 0 for i in all_df['name']]
    all_df['digits_in_name'] = [sum(c.isdigit() for c in i) if pd.notnull(i) else 0 for i in all_df['name']]
    all_df['description_length'] = [len(i) if pd.notnull(i) else 0 for i in all_df['description']]
    return all_df

In [ ]:
user_table_df = pd.read_csv("/Users/vorsi/Dropbox/bot and agenda-setting/codes and data/raw data/xr_users_classified_0326_wprob.csv")
user_table_df = generate_features(user_table_df)
user_table_copy = user_table_df.copy()

trait_cols = ['statuses_count', 'followers_count', "friends_count", "favourites_count","listed_count","default_profile","geo_enabled","profile_use_background_image","protected", "verified"]
dl_trait_cols = ['statuses_count', 'followers_count', "friends_count", "favourites_count","listed_count","default_profile",
               "geo_enabled","profile_use_background_image",
                "protected", 
                "verified", 
               'tweet_freq','followers_growth','friends_growth', 'favourites_growth', 'listed_growth',
               'follower_friend_ratio', 'screen_name_length','digits_in_screen_name', 
               'name_length','digits_in_name', 'description_length']

non_trait_cols = [i for i in list(user_table_df) if i not in trait_cols]
non_trait_cols_dl = [i for i in list(user_table_df) if i not in dl_trait_cols]
boolean_features = ["default_profile","geo_enabled","profile_use_background_image","protected", "verified"]
boolean_rf = ["protected", "verified"]
user_table_df_new = user_table_df[trait_cols]
user_table_df_dl = user_table_df[dl_trait_cols]




In [ ]:
#user_table_df[['is_bot_dl_rough', 'is_bot_dl_prob', 'is_bot_rf_prob', 'is_bot_rf']]

In [ ]:
x_df_rf = user_table_df_new.fillna(0.0)
x_df_rf[boolean_features] = x_df_rf[boolean_features].replace("True ",1.0)
x_df_rf[boolean_features] = x_df_rf[boolean_features].replace("False ",0.0)
#x_df_rf[boolean_rf] = x_df_rf[boolean_rf].replace("True ",1.0)
#x_df_rf[boolean_rf] = x_df_rf[boolean_rf].replace("False ",0.0)
x_df_rf = x_df_rf.apply(lambda x: np.asarray(x).astype(np.float32))

In [ ]:
x_df_rf

In [ ]:
x_df = user_table_df_dl.fillna(0.0)
x_df[boolean_features] = x_df[boolean_features].replace("True ",1.0)
x_df[boolean_features] = x_df[boolean_features].replace("False ",0.0)
x_df = x_df.apply(lambda x: np.asarray(x).astype(np.float32))

In [ ]:
import joblib

In [ ]:
bot_rf = joblib.load("random_forest_model_20trait.joblib")

In [ ]:
#bot_model_dl = keras.models.load_model("./bot_identification/DL_model_010423")

bot_rf10 = joblib.load("/Users/vorsi/Dropbox/bot and agenda-setting/codes and data/bot_identification_code_and_data/random_forest_model_10trait.joblib")

In [ ]:
bot_or_not_rf = bot_rf10.predict(x_df_rf)
bot_or_not_rf20 = bot_rf.predict(x_df)

In [ ]:
bot_or_not_rf20_prob = bot_rf.predict_proba(x_df)

In [ ]:
bot_or_not = bot_model_dl.predict(x_df)

In [ ]:
bot_or_not_ls = bot_or_not.tolist()

In [ ]:
bot_or_not_ls = [i[0] for i in bot_or_not_ls]

In [ ]:
plt.hist(bot_or_not_ls)

In [ ]:
bot_or_not_binary = [1 if i >=0.5 else 0 for i in bot_or_not_ls]

In [ ]:
user_table_df_new['is_bot_dl_rough'] = bot_or_not_binary
user_table_df_new['is_bot_dl_prob'] = bot_or_not_ls

#user_table_df_new.to_sql(table_name, if_exists = 'replace', index = False)

In [ ]:
bot_or_not_rf = bot_rf.predict(x_df_rf)

In [ ]:
len([i for i in bot_or_not_rf20 if i==1])

In [ ]:
bot_or_not_rf20_prob = [i[1] for i in bot_or_not_rf20_prob] 
#Returned a result in the format of [human_prob, bot_prob], we used the bot prob

In [ ]:
user_table_df_new['is_bot_rf20'] = bot_or_not_rf20
user_table_df_new['is_bot_rf20_prob'] = bot_or_not_rf20_prob
user_table_df_new['is_bot_rf10'] = bot_or_not_rf

In [ ]:

user_table_df_new = user_table_df_new.applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x)
user_table_df_new = user_table_df_new.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)
user_table_df_new = user_table_df_new.applymap(lambda x: x.replace('\t', '') if isinstance(x, str) else x)
user_table_df_new = user_table_df_new.applymap(lambda x: x.replace('\s', '') if isinstance(x, str) else x)

In [ ]:
user_table_df_new = user_table_df_new.applymap(lambda x: x.replace('\r', '') if isinstance(x, str) else x)

In [ ]:
user_table_df_new.to_csv("xr_users_classified_0422_wprob.csv", index = False, sep = ",")

In [ ]:
user_table_df_new[non_trait_cols] = user_table_df[non_trait_cols]

In [ ]:
user_table_df_new[user_table_df_new.is_bot_rf20==1].shape[0]

### Comparing

In [ ]:
corr = user_table_df_new.groupby(['is_bot_rf10','is_bot_rf20']).size().reset_index()

In [ ]:
corr

In [ ]:
user_table_df_new.groupby(['is_bot_dl_rough','is_bot_rf20']).size().reset_index()